In [ ]:
from libraries import *
from parameters import *
import statsmodels.api as sm
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

In [ ]:
os.getcwd()
os.chdir(projectDir)

In [ ]:
adata = sc.read(par_save_filename_5)
adata.X.shape

In [ ]:
adata.uns['Control_guides'] = [ x for x in adata.uns['feature_barcode_names'] if
                               (x.startswith("ONE_NONGENE_SITE") or x.startswith("NO_TARGET"))]
controlGM = adata.obs[adata.uns['Control_guides']]
controlGM =controlGM[controlGM.sum(axis=1) == 1]
adataControl = adata[controlGM.index,]
adataControl.obs['Control_guide'] = adataControl.obs[adataControl.uns['Control_guides']].idxmax(axis=1)
adataControl.shape

In [ ]:
sc.tl.pca(adataControl, use_highly_variable=True,n_comps=100, svd_solver='arpack')

In [ ]:
expressionMatrix = adataControl.obsm['X_pca']
guideMatrix = adataControl.obs[adataControl.uns['Control_guides']]

In [ ]:
pvalDF = pd.DataFrame()
coefDF = pd.DataFrame()
    
for j in range(0,100):
    print(j)
    est = sm.OLS(np.array(expressionMatrix)[:,j], np.array(guideMatrix))
    est2 = est.fit().summary2().tables[1]
    pvalDF[str(j)] = est2['P>|t|']
    coefDF[str(j)] = est2['Coef.']
    

In [ ]:
coefDF.index = adataControl.uns['Control_guides']

In [ ]:
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(coefDF)
k = pd.Series(coefDF.index[yhat == -1,])

In [ ]:
ee = EllipticEnvelope(contamination=0.1)
yhat = ee.fit_predict(coefDF)

k = k.append(pd.Series(coefDF.index[yhat == -1,]))

In [ ]:
lof = LocalOutlierFactor()
yhat = lof.fit_predict(coefDF)
k = k.append(pd.Series(coefDF.index[yhat == -1,]))

In [ ]:
ee = OneClassSVM(nu=0.01)
yhat = ee.fit_predict(coefDF)
k = k.append(pd.Series(coefDF.index[yhat == -1,]))

In [ ]:
allSel = pd.DataFrame(k.value_counts())
allSel.columns = ["nClassf"]

In [ ]:
allSel = allSel.loc[ allSel.nClassf > 2,]
allSel['OutlierGuides'] = allSel.index

In [ ]:
allSel = allSel.sort_values(by=['OutlierGuides'])

In [ ]:
allSel.loc[:,"OutlierGuides"].to_csv(par_outlier_controlguides_file, index=False)